### HW04: Practice with feature engineering, splitting data, and fitting and regularizing linear models

Waleed Almousa


### Hello Students:

- Start by downloading HW04.ipynb from this folder. Then develop it into your solution.
- Write code where you see "... your code here ..." below.
  (You are welcome to use more than one cell.)
- If you have questions, please ask them in class or office hours. Our TA
  and I are very happy to help with the programming (provided you start early
  enough, and provided we are not helping so much that we undermine your learning).
- When you are done, run these Notebook commands:
  - Shift-L (once, so that line numbers are visible)
  - Kernel > Restart and Run All (run all cells from scratch)
  - Esc S (save)
  - File > Download as > HTML
- Turn in:
  - HW04.ipynb to Canvas's HW04.ipynb assignment
  - HW04.html to Canvas's HW04.html assignment
  - As a check, download your files from Canvas to a new 'junk' folder. Try 'Kernel > Restart
  and Run All' on the '.ipynb' file to make sure it works. Glance through the '.html' file.
- Turn in partial solutions to Canvas before the deadline. e.g. Turn in part 1,
  then parts 1 and 2, then your whole solution. That way we can award partial credit
  even if you miss the deadline. We will grade your last submission before the deadline.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import accuracy_score


## 1. Feature engineering (one-hot encoding and data imputation)

(Note: This paragraph is not instructions but rather is to communicate context for this exercise. We use the same Titanic data we used in HW02:
- There we used `df.dropna()` to drop any observations with missing values; here we use data imputation instead.
- There we manually did one-hot encoding of the categorical `Sex` column by making a `Female` column; here we do the same one-hot encoding with the help of pandas's `df.join(pd.get_dummies())`.
- There we used a decision tree; here we use $k$-NN.

We evaluate how these strategies can improve model performance by allowing us to use columns with categorical or missing data.)

### 1a. Read the data from [http://www.stat.wisc.edu/~jgillett/451/data/kaggle_titanic_train.csv](http://www.stat.wisc.edu/~jgillett/451/data/kaggle_titanic_train.csv).
- Retain only these columns: Survived, Pclass, Sex, Age, SibSp, Parch.
- Display the first 7 rows.

These data are described at [https://www.kaggle.com/competitions/titanic/data](https://www.kaggle.com/competitions/titanic/data) (click on the small down-arrow to see the "Data Dictionary"), which is where they are from.
- Read that "Data Dictionary" paragraph (with your eyes, not python) so you understand what each column represents.


In [2]:
raw_data=pd.read_csv("http://www.stat.wisc.edu/~jgillett/451/data/kaggle_titanic_train.csv")

In [3]:
df=raw_data[["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch"]]
df.head(7)

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,male,22.0,1,0
1,1,1,female,38.0,1,0
2,1,3,female,26.0,0,0
3,1,1,female,35.0,1,0
4,0,3,male,35.0,0,0
5,0,3,male,NaN,0,0
6,0,1,male,54.0,0,0


### 1b. Try to train a $k$NN model to predict $y=$ 'Survived' from $X=$ these features: 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch'.
- Use $k = 3$ and the (default) euclidean metric.
- Notice at the bottom of the error message that it fails with the error "ValueError: could not convert string to float: 'male'".
- Comment out your .fit() line so the cell can run without error.

In [4]:

clf=KNeighborsClassifier(n_neighbors=3)
X=df[["Pclass", "Sex", "Age", "SibSp", "Parch"]]
y=df.Survived

# clf.fit(X, y)

### 1c. Try to train again, this time without the 'Sex' feature.
- Notice that it fails because "Input contains NaN".
- Comment out your .fit() line so the cell can run without error.
- Run `X.isna().any()` (where X is the name of your DataFrame of features) to see that
  the 'Age' feature has missing values. (You can see the first missing value in
  the sixth row that you displayed above.)

In [5]:

clf=KNeighborsClassifier(n_neighbors=3)
X=df[["Pclass", "Age", "SibSp", "Parch"]]
y=df.Survived

X.isna().any()
# clf.fit(X, y)

Pclass    False
Age        True
SibSp     False
Parch     False
dtype: bool

### 1d. Train without the 'Sex' and 'Age' features.
- Report accuracy on the training data with a line of the form
  `Accuracy on training data is  0.500` (0.500 may not be correct).

In [6]:
clf=KNeighborsClassifier(n_neighbors=3)
X=df[["Pclass", "SibSp", "Parch"]]
y=df.Survived
clf.fit(X, y)
training_acc=clf.score(X, y)
print(f"Accuracy on training data is {training_acc:.3}")

Accuracy on training data is 0.664


### 1e.  Use one-hot encoding
to include a binary 'male'  feature made from the 'Sex' feature. (Or include a binary 'female'
feature, according to your preference. Using both is unnecessary since either is the logical
negation of the other.) That is, train on these features: 'Pclass', 'SibSp', 'Parch', 'male'.
- Use pandas's df.join(pd.get_dummies())`.
- Report training accuracy as before.

In [7]:
try:
    df=df.join(pd.get_dummies(df.Sex, prefix="male"))
except:
    pass

clf=KNeighborsClassifier(n_neighbors=3)
X=df[["Pclass", "SibSp", "Parch", "male_male"]]
y=df.Survived
clf.fit(X, y)

training_acc=clf.score(X, y)
print(f"Accuracy on training data is {training_acc:.3}")

Accuracy on training data is 0.744


### 1f. Use data imputation
to include an 'age' feature made from 'Age' but replacing each missing value with the median
of the non-missing ages. That is, train on these features: 'Pclass', 'SibSp', 'Parch', 'male',
'age'.

- Report training accuracy as before.

In [8]:
imp = SimpleImputer(missing_values=np.nan, strategy='median', fill_value=None)
df["age"]=imp.fit_transform(df[['Age']])
clf=KNeighborsClassifier(n_neighbors=3)
X=df[["Pclass", "SibSp", "Parch", "male_male", "age"]]
y=df.Survived
clf.fit(X, y)

training_acc=clf.score(X, y)
print(f"Accuracy on training data is {training_acc:.3}")

Accuracy on training data is 0.863


## 2. Explore model fit, overfit, and regularization in the context of multiple linear regression

### 2a. Prepare the data:
- Read [http://www.stat.wisc.edu/~jgillett/451/data/mtcars.csv](http://www.stat.wisc.edu/~jgillett/451/data/mtcars.csv) into a DataFrame.
- Set a variable `X` to the subset consisting of all columns except `mpg`.
- Set a variable `y` to the `mpg` column.
- Use `train_test_split()` to split `X` and `y` into `X_train`, `X_test`, `y_train`, and `y_test`.
  - Reserve half the data for training and half for testing.
  - Use `random_state=0` to get reproducible results.

In [9]:
raw_data=pd.read_csv("http://www.stat.wisc.edu/~jgillett/451/data/mtcars.csv")

In [10]:
X=raw_data.drop(columns=["mpg", "Unnamed: 0"])
y=raw_data.mpg
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.5, random_state=0)

### 2b. Train three models on the training data and evaluate each on the test data:
- `LinearRegression()`
- `Lasso()`
- `Ridge()`

The evaluation consists in displaying MSE$_\text{train}, $ MSE$_\text{test}$, and the coefficients $\mathbf{w}$ for each model.

In [11]:
LR= linear_model.LinearRegression()
LR.fit(X_train, y_train)
LR_MSE_train = (1/y_train.size) * np.sum((y_train - LR.predict(X_train))**2)
LR_MSE_test = (1/y_test.size)  * np.sum((y_test - LR.predict(X_test))**2)
LR_w=LR.coef_

print(f"LINEAR REGRESSION MSE_train: {LR_MSE_train:.3}")
print(f"LINEAR REGRESSION MSE_test: {LR_MSE_test:.3}")
print(f"LINEAR REGRESSION coef_: {LR_w}")

lasso= linear_model.Lasso()
lasso.fit(X_train, y_train)
lasso_MSE_train = (1/y_train.size) * np.sum((y_train - lasso.predict(X_train))**2)
lasso_MSE_test = (1/y_test.size)  * np.sum((y_test - lasso.predict(X_test))**2)
lasso_w=lasso.coef_

print(f"\nLASSO MSE_train: {lasso_MSE_train:.3}")
print(f"LASSO MSE_test: {lasso_MSE_test:.3}")
print(f"LASSO coef_: {lasso_w}")

ridge= linear_model.Ridge()
ridge.fit(X_train, y_train)
ridge_MSE_train = (1/y_train.size) * np.sum((y_train - ridge.predict(X_train))**2)
ridge_MSE_test = (1/y_test.size)  * np.sum((y_test - ridge.predict(X_test))**2)
ridge_w=ridge.coef_

print(f"\nRIDGE MSE_train: {ridge_MSE_train:.3}")
print(f"RIDGE MSE_test: {ridge_MSE_test:.3}")
print(f"RIDGE coef_: {ridge_w:}")


LINEAR REGRESSION MSE_train: 0.386
LINEAR REGRESSION MSE_test: 30.2
LINEAR REGRESSION coef_: [ 1.08241627  0.02500827  0.02593759  3.12997296 -7.34326296  3.93233873
 -4.08551838 -1.2164054   5.98718293 -0.77097967]

LASSO MSE_train: 5.69
LASSO MSE_test: 13.0
LASSO coef_: [-0.         -0.03718773 -0.01681757  0.         -0.          0.
  0.          0.          0.         -0.84712891]

RIDGE MSE_train: 1.99
RIDGE MSE_test: 11.2
RIDGE coef_: [-0.01392446 -0.00744628  0.00400324  0.79271685 -3.21854592  1.09825372
 -0.48236992  0.47079795  1.49546846 -1.04547895]


### 2c. Answer a few questions about the models:
- Which one best fits the training data?
- Which one best fits the test data?
- Which one does feature selection by setting most coefficients to zero?


Linear regression best fits the training data

Ridge best fits the test data

Lasso does feature selection by setting most coefficients to zero
